In [1]:
1 + 1

2

In [ ]:
# スピン値(配列で表示)
σ = [-1, 1]

# ハミルトニアンの計算（隣接ペアの和）
function energy(config, L, J)　#config:スピン配置を1次元配列として渡す
    E = 0.0 #全体のエネルギーを初期化
    for x in 1:L
        for y in 1:L
            s = config[(x-1)*L + y]
            # 右隣（周期境界なし）
            if y < L
                s_r = config[(x-1)*L + y + 1]
                E -= J * s * s_r
            end
            # 下隣
            if x < L
                s_d = config[x*L + y]
                E -= J * s * s_d
            end
        end
    end
    return E
end

# 分配関数を計算
function partition_function(L; β=1.0, J=1.0)
    Z = 0.0
    N = L * L
    # 全スピン状態列挙（2^(L^2)）
    for i in 0:(2^N - 1)
        config = zeros(Int, N)
        for k in 1:N
            # ビットを使ってスピン割り当て
            config[k] = σ[bitstring(i)[end-k+1] == '1' ? 2 : 1]
        end
        E = energy(config, L, J)
        Z += exp(-β * E)
    end
    return Z
end

# 例：L=2, β=0.5, J=1.0
println("Z =", partition_function(2, β=0.5, J=1.0))

Z =27.048782764334526
